In [1]:
import numpy as np
from pathlib import Path
import sys
from timeit import default_timer as timer
from argparse import ArgumentParser
from cxiapi import cxiData
from cxiapi.litPixelsAnalyzer import litPixelsAnalyzer

In [2]:
# Experiment run number.
runs = [366]
# Hitfinding on which module.
module = 15
# Number of CPU processes to use, '0' means using all.
nproc = 0
# The folder of cxi files
cxi_folder = '/gpfs/exfel/u/scratch/SPB/202130/p900201/spi-comission/vds/'
# Cheetah files folder for calibration
calib_folder = '/gpfs/exfel/exp/SPB/202130/p900201/usr/Software/calib/r0361-r0362-r0363/'

In [3]:
for run in runs:
    print(f'run: {run}')
    print(f'hitfinding with module {module}\n')
    
    start = timer()
    cxi_path = Path(cxi_folder, f'r{run:04}.cxi')
    fn = str(cxi_path)
    with cxiData(fn, verbose=1, debug=0) as cxi:
        pulse = np.arange(0, 352)
        base_pulse_filter = np.ones(600, dtype="bool")
        base_pulse_filter[len(pulse):] = False
        base_pulse_filter[0] = False
        base_pulse_filter[18::32] = False
        base_pulse_filter[29::32] = False
        good_cells = pulse[base_pulse_filter[:len(pulse)]]
        cxi.setGoodCells(good_cells)

        cxi.setCalib(
            calib_folder
        )
        cxi.setGeom()

        start_litpixel = timer()
        analyzer = litPixelsAnalyzer(cxi, verbose=1)
        analyzer.getLitPixels(module, nproc=nproc)
        analyzer.writeLitPixels()
        end = timer()
        print('getLitPixels: {:.2f} s elapsed'.format(end - start_litpixel))
    end = timer()
    print('Run {:04} done: {:.2f} s elapsed'.format(run, end - start))

run: 366
hitfinding with module 15

VDS file contains 62304 shots
Module 0 contains 352 cells
16 calibration files found
VDS file: /gpfs/exfel/u/scratch/SPB/202130/p900201/spi-comission/vds/r0366.cxi
Calculating number of lit pixels for 58233 frames
Using 72 processes
Doing 1 chunks of 4096 frames each


100%|██████████| 72/72 [01:58<00:00,  1.65s/it] 

Writting to r0366_hits.h5
Writting to r0366_hits.h5 done
getLitPixels: 120.20 s elapsed
Run 0366 done: 120.33 s elapsed
